# సెషన్ 1 – చాట్ బూట్‌స్ట్రాప్ (ఫౌండ్రీ లోకల్)

ఈ నోట్‌బుక్ ఫౌండ్రీ లోకల్‌ను బూట్‌స్ట్రాప్ చేస్తుంది, ప్రాధాన్యత ఉన్న మోడల్ అలియాస్‌ను డౌన్‌లోడ్ చేస్తుంది, మరియు ఒక సాధారణ మరియు స్ట్రీమింగ్ చాట్ కంప్లీషన్‌ను నిర్వహిస్తుంది.


# పరిస్థితి
ఈ సెషన్ లో ఫౌండ్రీ లోకల్ ద్వారా స్థానిక చిన్న భాషా మోడల్ స్పందించడానికి అవసరమైన కనిష్టం పరిచయం చేయబడుతుంది. మీరు:
- SDK / క్లయింట్ డిపెండెన్సీలను ఇన్‌స్టాల్ చేయండి.
- ఎంచుకున్న అలియాస్ కోసం ఫౌండ్రీ లోకల్ మేనేజర్‌ను ప్రారంభించండి (డిఫాల్ట్: `phi-4-mini`).
- మోడల్ మెటాడేటాలో ఐచ్ఛిక ఫీల్డ్స్‌ను సహించడానికి రక్షణాత్మక మంకీ-ప్యాచ్‌ను వర్తింపజేయండి.
- ఒక సాధారణ చాట్ కంప్లీషన్ అభ్యర్థనను పంపండి.
- టోకెన్-ద్వారా-టోకెన్ స్పందనను స్ట్రీమ్ చేయండి.

లక్ష్యం RAG, రౌటింగ్ లేదా ఏజెంట్లకు వెళ్లే ముందు మీ స్థానిక రన్‌టైమ్ & నెట్‌వర్క్ మార్గాన్ని ధృవీకరించడం.


### వివరణ: డిపెండెన్సీ ఇన్‌స్టాలేషన్
ఈ కనిష్ట చాట్ ఫ్లో కోసం అవసరమైన Python ప్యాకేజీలను ఇన్‌స్టాల్ చేస్తుంది:
- `foundry-local-sdk`: స్థానిక మోడల్స్ మరియు సర్వీస్ లైఫ్‌సైకిల్‌ను నిర్వహించండి.
- `openai`: చాట్ కంప్లీషన్స్ కోసం పరిచయమైన క్లయింట్ అభిసరణ.
- `rich`: స్పష్టమైన నోట్‌బుక్ అవుట్‌పుట్ కోసం అందమైన ప్రింటింగ్.

మళ్లీ నడపడం సురక్షితం (ఐడెంపోటెంట్). మీ వాతావరణంలో ఇవి ఇప్పటికే ఉంటే దాటవేయండి.


In [1]:
# Install required libraries (idempotent)
%pip install -q foundry-local-sdk openai rich


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### వివరణ: కోర్ ఇంపోర్ట్స్
నోట్‌బుక్‌లో మొత్తం ఉపయోగించే మాడ్యూల్స్‌ను తీసుకువస్తుంది:
- స్థానిక మోడల్ రన్‌టైమ్‌తో ఇంటర్‌ఫేస్ చేయడానికి `FoundryLocalManager`.
- మనకు పరిచయమైన చాట్ కంప్లీషన్ API సర్ఫేస్‌ను మళ్లీ ఉపయోగించడానికి `OpenAI` క్లయింట్.
- స్టైల్డ్ అవుట్‌పుట్ కోసం `rich.print`.

ఇక్కడ ఎటువంటి నెట్‌వర్క్ కాల్స్ జరగవు—ఇది కేవలం నేమ్‌స్పేస్‌ను సిద్ధం చేస్తుంది.


In [2]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI
from rich import print

### వివరణ: మేనేజర్ ప్రారంభం & మెటాడేటా ప్యాచ్  
ఎంచుకున్న అలియాస్ కోసం `FoundryLocalManager` ను ప్రారంభించి, `promptTemplate` null కావచ్చు అనే పరిస్థితుల్లో సర్వీస్ ప్రతిస్పందనలను సున్నితంగా నిర్వహించడానికి రక్షణాత్మక మంకీ‑ప్యాచ్‌ను వర్తింపజేస్తుంది.

ప్రధాన ఫలితాలు:  
- సర్వీస్ స్థితి మరియు ఎండ్‌పాయింట్‌ను నిర్ధారిస్తుంది.  
- క్యాష్ చేసిన మోడల్స్ జాబితా (లోకల్ స్టోర్‌ను ధృవీకరిస్తుంది).  
- అలియాస్ కోసం ఖచ్చితమైన మోడల్ IDని పరిష్కరిస్తుంది (తరువాత చాట్ కాల్స్‌లో ఉపయోగించబడుతుంది).  

మీరు రా సర్వీస్ మెటాడేటాలో ధృవీకరణ సమస్యలను ఎదుర్కొంటే, ఈ నమూనా SDKని ఫోర్క్ చేయకుండా ఎలా శుభ్రపరచాలో చూపిస్తుంది.


In [3]:
# Monkeypatch to tolerate service responses where promptTemplate is null
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            # Normalize to empty dict so pydantic validation passes
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"[yellow]Warning: safe wrapper encountered issue normalizing promptTemplate: {e}[/yellow]")
    return _original_from_list_response(response)

# Apply patch only once
if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore

ALIAS = os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini')
manager = FoundryLocalManager(ALIAS)
print(f'[bold green]Service running:[/bold green] {manager.is_service_running()}')
print(f'Endpoint: {manager.endpoint}')
print('Cached models:', manager.list_cached_models())
model_id = manager.get_model_info(ALIAS).id
print(f'Using model id: {model_id}')

Service running: True

Endpoint: http://127.0.0.1:50262/v1

Cached models:
[
    FoundryModelInfo(
        alias='phi-4-mini',
        id='Phi-4-mini-instruct-generic-gpu:4',
        version='4',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-4-mini-instruct-generic-gpu/versions/4',
        file_size_mb=3809,
        prompt_template={
            'system': '<|system|>{Content}<|end|>',
            'user': '<|user|>{Content}<|end|>',
            'assistant': '<|assistant|>{Content}<|end|>',
            'prompt': '<|user|>{Content}<|end|><|assistant|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-0.5b',
        id='qwen2.5-0.5b-instruct-generic-gpu:3',
        version='3',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-0.5b-instruct-generic-gpu/versions/3',
        file_size_mb=700,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-3.5-mini',
        id='Phi-3.5-mini-instruct-generic-gpu:1',
        version='1',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-3.5-mini-instruct-generic-gpu/versions/1',
        file_size_mb=2211,
        prompt_template={
            'prompt': '<|user|>\n{Content}<|end|>\n<|assistant|>',
            'assistant': '<|assistant|>\n{Content}<|end|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    )
]

Using model id: Phi-4-mini-instruct-generic-gpu:4

### వివరణ: ప్రాథమిక చాట్ పూర్తి
స్థానిక Foundry ఎండ్‌పాయింట్‌ను సూచించే `OpenAI`-అనుకూల క్లయింట్‌ను సృష్టించి ఒకే ఒక నాన్-స్ట్రీమింగ్ చాట్ పూర్తి చేస్తుంది. ఇక్కడ దృష్టి పెట్టండి:
- మోడల్ తప్పులేకుండా స్పందించిందని నిర్ధారించండి.
- ఆలస్యం / అవుట్‌పుట్ ఫార్మాట్‌ను ధృవీకరించండి.
- వనరులను ఆదా చేయడానికి `max_tokens` ను తక్కువగా ఉంచండి.

ఇది విఫలమైతే, Foundry Local సేవ నడుస్తుందో లేదో మరియు అలియాస్ సరిగ్గా పరిష్కరించబడిందో మళ్లీ తనిఖీ చేయండి.


In [4]:
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
prompt = 'List two benefits of local inference for privacy.'
resp = client.chat.completions.create(
    model=model_id,
    messages=[{'role':'user','content':prompt}],
    max_tokens=120,
    temperature=0.5
)
print(resp.choices[0].message.content)

Local inference for privacy refers to the practice of performing data analysis on a local device without sending 
sensitive information to a central server. Two benefits of this approach are:


1. **Enhanced Privacy**: Local inference keeps personal data on the user's device, reducing the risk of data 
breaches and unauthorized access. Since the data is not transmitted over the network, it is less susceptible to 
interception by malicious actors.


2. **Data Sovereignty**: Users retain control over their data, as it does not leave their device. This means that 
individuals or organizations can comply with local data protection regulations, such as the General

### వివరణ: స్ట్రీమింగ్ చాట్ పూర్తి
మెరుగైన అనుభూతి ఆలస్యం మరియు ఇంటరాక్టివ్ UX కోసం టోకెన్ స్ట్రీమింగ్‌ను ప్రదర్శిస్తుంది. లూప్ వచ్చినట్లుగా పెరుగుతున్న డెల్టాలను ముద్రిస్తుంది:
- ప్రారంభ భాగపు అవుట్పుట్ ముఖ్యం అయ్యే చాట్ UIలకు ఉపయోగకరం.
- పూర్తి పూర్తి ఆలస్యం తో టోకెన్ థ్రూపుట్‌ను కొలవడానికి అనుమతిస్తుంది.

మీరు ఈ నమూనాను టోకెన్లను సేకరించడానికి, ప్రోగ్రెస్ విడ్జెట్‌ను నవీకరించడానికి లేదా మధ్యలో జనరేషన్‌ను రద్దు చేయడానికి అనుకూలపరచుకోవచ్చు.


In [5]:
# Streaming example
stream = client.chat.completions.create(
    model=model_id,
    messages=[{'role':'user','content':'Give a one-sentence definition of edge AI.'}],
    stream=True,
    max_tokens=60,
    temperature=0.4
)
for chunk in stream:
    delta = chunk.choices[0].delta
    if delta and delta.content:
        print(delta.content, end='', flush=True)
print()

Edge

AI

refers

to

artificial

...

latency

and

bandwidth

usage

.

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో పొరపాట్లు లేదా తప్పిదాలు ఉండవచ్చు. మూల పత్రం దాని స్వదేశీ భాషలోనే అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారానికి, ప్రొఫెషనల్ మానవ అనువాదం సిఫార్సు చేయబడుతుంది. ఈ అనువాదం వాడకంలో ఏర్పడిన ఏవైనా అపార్థాలు లేదా తప్పుదారుల కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
